<a href="https://colab.research.google.com/github/Vishnu473/web-scrapping-project/blob/main/WebScrappingProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests # installing requests

In [ ]:
!pip install beautifulsoup4 # installing requests

In [ ]:
# Checking if the request was success

import requests

url = 'https://traveltriangle.com/blog/places-to-visit-in-india-before-you-turn-30/'  # Replace with your target website
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    html_content = response.text
    # print(html_content)
    print("HTML content fetched successfully!")
else:
    print(f"Failed to fetch page. Status code: {response.status_code}")


In [ ]:
from bs4 import BeautifulSoup
import requests
import re

main_url = 'https://www.holidify.com/country/india/places-to-visit.html'

to_visit_in_India = []

for page_number in range(0, 3):

    url = f'{main_url}?pageNum={page_number}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')


    # Extract places
    places = soup.find_all('div', class_='col-12 col-md-6 pr-md-3')

    for place in places:
        name = place.find('h3', class_='card-heading').text.split('.')[1].strip()
        img = place.find('img',class_='card-img-top')['data-original']
        images = [img]
        # Extract data-original URLs from lazyBG divs
        imgs = place.find_all('div', class_='lazyBG card-img-top')

        for image in imgs:
            data_original_url = image.get('data-original')
            if data_original_url:
                images.append(data_original_url)

        link = place.find('a')['href']

        # Getting the detail from link
        detail_url = f'https://www.holidify.com{link}'
        link = detail_url
        rating = place.find('span', class_='rating-badge').text
        info = place.find('p', class_='card-text').text.strip().replace('\xa0', ' ')
        time = place.find('p', class_='mb-3').text.split(':')[1]
        time = re.split(r'[;,]', time)
        time = [t.strip() for t in time if t.strip()]
        more_places = place.find('p', class_='objective').text
        no_places = more_places.strip().split()
        if no_places[-3].isdigit():
          tourist_attractions_count = " ".join(no_places[-3:])
        else:
          tourist_attractions_count = "-"

        to_visit_in_India.append({
            'name': name,
            'image': images,
            'rating': rating.strip().split('/')[0].strip(),
            'link': link,
            'info': info,
            'time': time,
            'tourist_attractions_count': tourist_attractions_count
        })
print(to_visit_in_India)


In [ ]:
# Saving the output in places_data json file

import json

places_data = to_visit_in_India

json_file_name = 'places_data.json'

# Write to JSON file
with open(json_file_name, 'w') as json_file:
    json.dump(places_data, json_file, indent=4)
print(f'Data successfully written to {json_file_name}')
